In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### 구조적 스트리밍
- [예제] 소매 데이터 (by-day)
- 먼저 일반 csv file loading 후 분석

In [2]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("c:/SparkDG/data/retail-data/by-day/*.csv")     #여러 csv 파일을 함께 DataFrame에 loading함

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [3]:
# 하루를 기준으로 고객 ID 별 총 구매비용 내림차순 출력
from pyspark.sql.functions import window, column, desc, col
staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .sort(desc("sum(total_cost)"))\
  .show(5)


+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   17450.0|[2011-09-20 09:00...|          71601.44|
|      null|[2011-11-14 09:00...|          55316.08|
|      null|[2011-11-07 09:00...|          42939.17|
|      null|[2011-03-29 09:00...| 33521.39999999998|
|      null|[2011-12-08 09:00...|31975.590000000007|
+----------+--------------------+------------------+
only showing top 5 rows



In [5]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

### 스트리밍 데이터 분석
- 여기서는 csv 파일을 스트리밍 처럼 만든 후 분석

In [6]:
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\              # 한 번에 파일 1개씩 읽음
    .format("csv")\
    .option("header", "true")\
    .load("c:/SparkDG/data/retail-data/by-day/*.csv")

In [7]:
#DataFrame이 스트리밍인지 확인
streamingDataFrame.isStreaming

True

In [13]:
#하루를 기준으로 고객 ID 별 총 구매비용 내림차순 정렬 (지연 연산)
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

#스트리밍 액션 호출 / memory: 인메모리 테이블에 저장 / 저장될 테이블 명 / 모든 수행결과를 테이블에 저장
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

In [14]:
#쿼리 실행 결과 확인
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|      null|[2010-12-21 09:00...|31347.479999999938|
|   18102.0|[2010-12-07 09:00...|          25920.37|
|      null|[2010-12-10 09:00...|25399.560000000012|
|      null|[2010-12-17 09:00...|25371.769999999768|
|      null|[2010-12-06 09:00...|23395.099999999904|
+----------+--------------------+------------------+
only showing top 5 rows



### 머신러닝
- MLlib을 이용하여 데이터 전처리, 모델 학습, 예측 가능
- 분류, 회귀, 군집화, 딥러닝 등 다양한 API 제공

### 저수준 API
- DataFrame 연산도 사실 RDD를 기반으로 만들어졌음
- 현재는 대부분 RDD를 사용하지 않고 구조적 API를 사용함
- 비정형 데이터나 정제되지 않은 원시 데이터 처리시에는 RDD를 사용해야함